In [6]:
from datasets import load_dataset
import pandas as pd
dataset = load_dataset("arbml/Tunisian_Dialect_Corpus")
df = pd.DataFrame(dataset['train'])
df.to_csv('tunisian_dialect_corpus.csv', index=False)

Repo card metadata block was not found. Setting CardData to empty.


In [3]:
df

,Tweet,label
0,Nn mouch 7louwa faza,1
1,mabladkom 3bed tfouuuuh,1
2,تفووووووه عليك و علا والديك على عايلتك و على ا...,1
3,لا يليق بهذا البرنامج,1
4,رهدان,1
...,...,...
49884,الله يستر ن,0
49885,الله يستر ن,0
49886,ربي اكون فى عونكم بالحق ربي ابقي الستر على تون...,0
49887,,0


In [7]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("UBC-NLP/MARBERT")

lengths = []
for text in df["Tweet"]:
    lengths.append(len(tokenizer(text).input_ids))

np.percentile(lengths, [50, 75, 90, 95, 99])


array([10., 17., 30., 42., 89.])

In [8]:
import re
import unicodedata

def normalize_text(text):
    # 1. Lowercase
    text = text.lower()

    # 2. Remove diacritics
    text = ''.join(c for c in unicodedata.normalize('NFD', text)
                   if unicodedata.category(c) != 'Mn')

    # 3. Normalize Arabic letters
    text = re.sub('[أإآ]', 'ا', text)
    text = text.replace('ى', 'ي')
    text = text.replace('ة', 'ه')

    # 4. Remove elongations (repeated letters >2)
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)  # e.g., مشششششكوور → مششكوور

    # 5. Arabizi number transliteration
    arabizi_map = {
        '2': 'ء',
        '3': 'ع',
        '5': 'خ',
        '6': 'ط',
        '7': 'ح',
        '8': 'ق',
        '9': 'ق'
    }
    for num, arabic_char in arabizi_map.items():
        text = text.replace(num, arabic_char)

    # 6. Remove URLs and mentions
    text = re.sub(r'http\S+|www.\S+', '', text)  # URLs
    text = re.sub(r'@\w+', '', text)             # mentions

    # 7. Remove extra spaces
    text = re.sub('\s+', ' ', text).strip()

    return text

# Apply to your dataset
df['Tweet'] = df['Tweet'].apply(normalize_text)
df['Tweet']

<>:38: SyntaxWarning: invalid escape sequence '\s'
<>:38: SyntaxWarning: invalid escape sequence '\s'
C:\Users\msi\AppData\Local\Temp\ipykernel_18644\334229453.py:38: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text).strip()


0                                     nn mouch حlouwa faza
1                                    mabladkom عbed tfouuh
2        تفووه عليك و علا والديك علي عايلتك و علي اصلك ...
3                                    لا يليق بهذا البرنامج
4                                                    رهدان
                               ...                        
49884                                          الله يستر ن
49885                                          الله يستر ن
49886    ربي اكون في عونكم بالحق ربي ابقي الستر علي تون...
49887                                                     
49888                                        ربي يلطف بينا
Name: Tweet, Length: 49889, dtype: object

In [10]:
import pandas as pd
import re

def validate_normalization(df, column='Tweet'):
    results = {}

    # 1. Check for remaining digits in words (Arabizi not converted)
    results['remaining_digits'] = df[column].str.contains(r'\d').sum()

    # 2. Check for unwanted Arabic letters that should have been normalized
    results['un_normalized_alef'] = df[column].str.contains(r'[أإآ]').sum()
    results['un_normalized_ya']  = df[column].str.contains(r'ى').sum()
    results['un_normalized_ta']  = df[column].str.contains(r'ة').sum()

    # 3. Check for multiple consecutive spaces
    results['extra_spaces'] = df[column].str.contains(r'\s{2,}').sum()

    # 4. Check for unwanted characters (not Arabic letters, Latin letters, digits, or basic punctuation)
    results['unwanted_chars'] = df[column].str.contains(r'[^ء-يA-Za-z0-9\s!?.,]').sum()

    # 5. Check for empty strings
    results['empty_strings'] = (df[column].str.strip() == '').sum()

    # 6. Sample some problematic rows for review
    sample_size = 50
    problematic_rows = df[df[column].str.contains(r'\d|[أإآ]|ى|ة|[^ء-يA-Za-z0-9\s!?.,]')].head(sample_size)

    # Convert results dictionary to DataFrame
    results_df = pd.DataFrame(list(results.items()), columns=['Issue', 'Count'])

    return results_df, problematic_rows

# Example usage
validation_df, sample_issues = validate_normalization(df, 'Tweet')

print("Validation Summary:")
print(validation_df)

print("\nSample problematic rows for inspection:")
print(sample_issues)
df

Validation Summary:
                Issue  Count
0    remaining_digits   2377
1  un_normalized_alef      0
2    un_normalized_ya      0
3    un_normalized_ta      0
4        extra_spaces      0
5      unwanted_chars  11168
6       empty_strings   1888

Sample problematic rows for inspection:
                                                 Tweet  label
8                            شنوي هذا شنيه هالمسخره ؟؟      1
12      nari عlik ya bledi .._x00d_ choft fik laعjayeb      1
22   borjouleya msatet l'emission dima politique po...      1
30                   مثقف بودورو . ممثل فاشل راك مقزز"      1
42                                      "تقول فرخ صغير      1
47                                     "مساط بفف_x00d_      1
67                                      blid barcha :p      1
86   عزيزه بولبيار حايره في عجب ربي درويش ومصطك _x00d_      1
87   ya si taoufik , :( rak maset laset , , thaقaft...      1
96                                       خرده ڨديمه هه      1
99   لاادعو الي مقاطعه ها

,Tweet,label
0,nn mouch حlouwa faza,1
1,mabladkom عbed tfouuh,1
2,تفووه عليك و علا والديك علي عايلتك و علي اصلك ...,1
3,لا يليق بهذا البرنامج,1
4,رهدان,1
...,...,...
49884,الله يستر ن,0
49885,الله يستر ن,0
49886,ربي اكون في عونكم بالحق ربي ابقي الستر علي تون...,0
49887,,0


In [11]:
import re
import unicodedata
import pandas as pd

def preprocess_text(text):
    if not isinstance(text, str):
        return ""

    # 1. Lowercase
    text = text.lower()

    # 2. Remove diacritics
    text = ''.join(c for c in unicodedata.normalize('NFD', text)
                   if unicodedata.category(c) != 'Mn')

    # 3. Normalize Arabic letters
    text = re.sub('[أإآ]', 'ا', text)
    text = text.replace('ى', 'ي')
    text = text.replace('ة', 'ه')

    # 4. Remove elongations (letters repeated more than twice)
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)

    # 5. Arabizi number transliteration
    arabizi_map = {
        '2': 'ء',
        '3': 'ع',
        '5': 'خ',
        '6': 'ط',
        '7': 'ح',
        '8': 'ق',
        '9': 'ق'
    }
    for num, arabic_char in arabizi_map.items():
        text = text.replace(num, arabic_char)

    # 6. Remove URLs and mentions
    text = re.sub(r'http\S+|www.\S+', '', text)  # URLs
    text = re.sub(r'@\w+', '', text)             # mentions

    # 7. Remove unwanted characters (keep Arabic, Latin letters, digits, basic punctuation)
    text = re.sub(r'[^ء-يA-Za-z0-9\s!?.,]', '', text)

    # 8. Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Apply preprocessing to your dataset
df['text'] = df['Tweet'].apply(preprocess_text)

# Remove empty tweets after preprocessing
df = df[df['text'].str.strip() != ''].reset_index(drop=True)

# Optional: run validation to check
validation_df, sample_issues = validate_normalization(df, column='Tweet')
print("Validation Summary:")
print(validation_df)

print("\nSample problematic rows (if any):")
print(sample_issues)


Validation Summary:
                Issue  Count
0    remaining_digits   2359
1  un_normalized_alef      0
2    un_normalized_ya      0
3    un_normalized_ta      0
4        extra_spaces      0
5      unwanted_chars  10695
6       empty_strings      0

Sample problematic rows (if any):
                                                 Tweet  label  \
8                            شنوي هذا شنيه هالمسخره ؟؟      1   
12      nari عlik ya bledi .._x00d_ choft fik laعjayeb      1   
22   borjouleya msatet l'emission dima politique po...      1   
30                   مثقف بودورو . ممثل فاشل راك مقزز"      1   
42                                      "تقول فرخ صغير      1   
47                                     "مساط بفف_x00d_      1   
67                                      blid barcha :p      1   
86   عزيزه بولبيار حايره في عجب ربي درويش ومصطك _x00d_      1   
87   ya si taoufik , :( rak maset laset , , thaقaft...      1   
96                                       خرده ڨديمه هه      1  